In [9]:
import pandas as pd 
from sklearn.model_selection import train_test_split
#Cargamos el Dataset
file_path = "dataset/dataset.csv"
df = pd.read_csv(file_path)


In [8]:
#Listamos columnas para X y con drop separamos como un filter
X_columns = [
    'latitude_pos_tx', 'longitude_pos_tx', 'elevation_pos_tx', 'fc', 'elevation', 
    'azimuth', 'year', 'mmdd', 'UTI', 'hour', 'delay', 'terrestrial_range', 
    'slant_range', 'final_latitude', 'final_longitude', 'final_elevation'
]
X = df[X_columns]
Y = df.drop(columns=X_columns)

##### División de los datos en conjuntos de entrenamiento y prueba
  `train_test_split` es una función que forma parte de la biblioteca de scikit-learn en el módulo `sklearn.model_selection`.
  Se lo usa comunmente para dividir un conjunto de datos en dos subconjuntos (entrenamiento y prueba)
#### Sus parámetros
* **Datos**(obligatorio): Conjunto de datos que se desea dividir. 
  ```python
    train_test_split(X,Y)
* **test_size**(opcional): Define el porcentaje o la cantidad de datos que se reserva para prueba. Si es 0.2 indica un 20% reservado para prueba.
  ```python
    train_test_split(X,Y, test_size=0.2)
* **train_size**(opcional): Especifica el tamaño del conjunto de Entrenamiento.
* **random_state**(opcional): Controla la aleatoridad de la división. Si se usa un valor fijo (`random_state=42`), la división siempre es la misma cada vez que se ejecute el código.
  ```python
    train_test_split(X, Y, test_size=0.2, random_state=42)
* **shuffle** (opcional): Especificsa si los datos deben ser barajados antes de ser divididos. Por defecto está en `True`. Es decir, los datos se mezclarán aleatoriamente antes de ser divididos.
  


In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

print(type(x_train))